In [7]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("delta").master("spark://spark-master:7077") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","datalake") \
    .config("spark.hadoop.fs.s3a.secret.key","datalake") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()

In [2]:
spark

In [3]:
spark.sql('show tables from default').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    teste|      false|
|  default|   teste2|      false|
|  default|     city|      false|
+---------+---------+-----------+



In [4]:
df = spark.read.csv("s3a://delta-lake/results.csv",header=True)

In [5]:
df.show()

+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|      date|   home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName| cityCity|     countryCountry|neutralTRUE|
+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|1872-11-30|        Scotland|      England|             0|             0|      Friendly|  Glasgow|           Scotland|      FALSE|
|1873-03-08|         England|     Scotland|             4|             2|      Friendly|   London|            England|      FALSE|
|1874-03-07|        Scotland|      England|             2|             1|      Friendly|  Glasgow|           Scotland|      FALSE|
|1875-03-06|         England|     Scotland|             2|             2|      Friendly|   London|            England|      FALSE|
|1876-03-04|        Scotland|      England|             3|             0|      Frie

In [6]:
df.write.csv('hdfs:/datalake/teste',mode='overwrite')

In [5]:
df = df.withColumnRenamed('date','dt')

In [6]:
df.show()

+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|        dt|   home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName| cityCity|     countryCountry|neutralTRUE|
+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|1872-11-30|        Scotland|      England|             0|             0|      Friendly|  Glasgow|           Scotland|      FALSE|
|1873-03-08|         England|     Scotland|             4|             2|      Friendly|   London|            England|      FALSE|
|1874-03-07|        Scotland|      England|             2|             1|      Friendly|  Glasgow|           Scotland|      FALSE|
|1875-03-06|         England|     Scotland|             2|             2|      Friendly|   London|            England|      FALSE|
|1876-03-04|        Scotland|      England|             3|             0|      Frie

In [8]:
df_jogosNoBrasil = df.filter(df.countryCountry == 'Brazil')

In [9]:
df_jogosNoBrasil.count()

529

In [10]:
df.count()

40839

In [4]:
import uuid
myuuid = uuid.uuid4()
path = ("s3a://delta-lake/demo/{}").format(myuuid)
print(path)

s3a://delta-lake/demo/d0b8b558-13b5-4ca4-b09c-7b441e483369


In [5]:
df.write.format("delta").mode('overwrite').save(path)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Jav

Py4JError: An error occurred while calling o57.save

In [6]:
df.printSchema()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
deltaTable = DeltaTable.forPath(spark, path)

In [ ]:
type(deltaTable)

In [ ]:
d = spark.read.format("delta").load(path)

In [ ]:
d.show()

In [ ]:
type(d)

In [ ]:
deltaTable.toDF().printSchema()

In [ ]:
d.filter(d.countryCountry == 'Brazil').show()

In [18]:
code = spark.read.csv('s3a://camada-bronze/country_code.csv',header=True)

In [19]:
code.show()

+-------------------+-------+-------+------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|               name|alpha-2|alpha-3|country-code|   iso_3166-2|  region|          sub-region|intermediate-region|region-code|sub-region-code|intermediate-region-code|
+-------------------+-------+-------+------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|        Afghanistan|     AF|    AFG|         004|ISO 3166-2:AF|    Asia|       Southern Asia|               null|        142|            034|                    null|
|      Åland Islands|     AX|    ALA|         248|ISO 3166-2:AX|  Europe|     Northern Europe|               null|        150|            154|                    null|
|            Albania|     AL|    ALB|         008|ISO 3166-2:AL|  Europe|     Southern Europe|               null|        150|            039|                  

In [21]:
df_jogosNoBrasil.join(code,df.countryCountry == code.name,'left').show()

+----------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+-----------+------+-------+-------+------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|        dt|home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName|      cityCity|countryCountry|neutralTRUE|  name|alpha-2|alpha-3|country-code|   iso_3166-2|  region|          sub-region|intermediate-region|region-code|sub-region-code|intermediate-region-code|
+----------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+-----------+------+-------+-------+------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|1919-05-11|       Brazil|        Chile|             6|             0|  Copa América|Rio de Janeiro|        Brazil|      FALSE|Brazil|     BR|    BRA|       

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable.alias("delta") \
  .merge(
    code.alias("code"),
    "delta.countryCountry = code.name") \
  .whenNotMatchedInsert(values = { "code": col("code.alpha-3") }) \
  .execute()

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("delta") \
  .merge(
    code.alias("code"),
    "delta.countryCountry = code.name") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

In [ ]:
data = spark.range(0, 5)
data.write.format("delta").save("s3a://tmp/delta-table")

In [ ]:
spark.sql('show databases').show()


In [12]:
df.createTempView('tab')

AnalysisException: Temporary view 'tab' already exists

In [16]:
spark.sql('select * from tab where countryCountry = "Brazil" order by dt desc').show()

+----------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+-----------+
|        dt|home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName|      cityCity|countryCountry|neutralTRUE|
+----------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+-----------+
|2019-07-07|       Brazil|         Peru|             3|             1|  Copa América|Rio de Janeiro|        Brazil|      FALSE|
|2019-07-06|    Argentina|        Chile|             2|             1|  Copa América|     São Paulo|        Brazil|       TRUE|
|2019-07-03|        Chile|         Peru|             0|             3|  Copa América|  Porto Alegre|        Brazil|       TRUE|
|2019-07-02|       Brazil|    Argentina|             2|             0|  Copa América|Belo Horizonte|        Brazil|      FALSE|
|2019-06-29|      Uruguay|         Peru|             0|             0|  Copa América|      Salvador|    

In [17]:
df.where(df.countryCountry == 'Brazil').orderBy('dt', ascending = False).show()

+----------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+-----------+
|        dt|home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName|      cityCity|countryCountry|neutralTRUE|
+----------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+-----------+
|2019-07-07|       Brazil|         Peru|             3|             1|  Copa América|Rio de Janeiro|        Brazil|      FALSE|
|2019-07-06|    Argentina|        Chile|             2|             1|  Copa América|     São Paulo|        Brazil|       TRUE|
|2019-07-03|        Chile|         Peru|             0|             3|  Copa América|  Porto Alegre|        Brazil|       TRUE|
|2019-07-02|       Brazil|    Argentina|             2|             0|  Copa América|Belo Horizonte|        Brazil|      FALSE|
|2019-06-29|      Uruguay|         Peru|             0|             0|  Copa América|      Salvador|    

In [2]:
import requests
import json

In [3]:
r = requests.get('https://www.boredapi.com/api/activity/')

In [4]:
r

<Response [200]>

In [5]:
rr = json.dumps(r.json())

In [6]:
rr

'{"activity": "Go to a concert with some friends", "type": "social", "participants": 4, "price": 0.6, "link": "", "key": "4558850", "accessibility": 0.4}'

In [7]:
df_json = spark.sparkContext.parallelize(rr)

In [10]:
df_json.collect()

['{',
 '"',
 'a',
 'c',
 't',
 'i',
 'v',
 'i',
 't',
 'y',
 '"',
 ':',
 ' ',
 '"',
 'G',
 'o',
 ' ',
 't',
 'o',
 ' ',
 'a',
 ' ',
 'c',
 'o',
 'n',
 'c',
 'e',
 'r',
 't',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 's',
 'o',
 'm',
 'e',
 ' ',
 'f',
 'r',
 'i',
 'e',
 'n',
 'd',
 's',
 '"',
 ',',
 ' ',
 '"',
 't',
 'y',
 'p',
 'e',
 '"',
 ':',
 ' ',
 '"',
 's',
 'o',
 'c',
 'i',
 'a',
 'l',
 '"',
 ',',
 ' ',
 '"',
 'p',
 'a',
 'r',
 't',
 'i',
 'c',
 'i',
 'p',
 'a',
 'n',
 't',
 's',
 '"',
 ':',
 ' ',
 '4',
 ',',
 ' ',
 '"',
 'p',
 'r',
 'i',
 'c',
 'e',
 '"',
 ':',
 ' ',
 '0',
 '.',
 '6',
 ',',
 ' ',
 '"',
 'l',
 'i',
 'n',
 'k',
 '"',
 ':',
 ' ',
 '"',
 '"',
 ',',
 ' ',
 '"',
 'k',
 'e',
 'y',
 '"',
 ':',
 ' ',
 '"',
 '4',
 '5',
 '5',
 '8',
 '8',
 '5',
 '0',
 '"',
 ',',
 ' ',
 '"',
 'a',
 'c',
 'c',
 'e',
 's',
 's',
 'i',
 'b',
 'i',
 'l',
 'i',
 't',
 'y',
 '"',
 ':',
 ' ',
 '0',
 '.',
 '4',
 '}']

In [11]:
import pandas 

ConnectionRefusedError: [Errno 111] Connection refused